In [1]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD
import sys
sys.path.append('../')

sys.path.append('../andre')
import andre.utils as utils
from andre.utils import schema as SCHEMA
import collections

In [2]:
publishers_btlf_graph = Graph()
publishers_btlf_graph.bind("schema","http://schema.org/", override=True, replace=True)
publishers_btlf_graph.parse("../final_datasets/btlf_books/Graphes/grapheEditeurs_Imprint_andre.ttl")
len(list(publishers_btlf_graph))

2044

In [16]:
publishers_bnf_constellations_graph = Graph()
publishers_bnf_constellations_graph.bind("schema","http://schema.org/", override=True, replace=True)
publishers_bnf_constellations_graph.parse("../final_datasets/publishers.ttl")
len(list(publishers_bnf_constellations_graph))

4690

In [4]:
publishers_bnf_constellations_dict = {}
doublons = collections.defaultdict(lambda: [])
for publisher in publishers_bnf_constellations_graph.subjects(RDF.type, SCHEMA.Publisher):
    raw_name = str(publishers_bnf_constellations_graph.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)

    doublons[preprocessed_name].append(publisher)
    
    if preprocessed_name not in publishers_bnf_constellations_dict.keys(): # don't add doublons
        # print("doublons", name)
        # doublons[preprocessed_name]+= publisher
        publishers_bnf_constellations_dict[preprocessed_name] = utils.Publisher(source="bnf_constellations", 
                                                                        uri=publisher, 
                                                                        raw_name=raw_name, 
                                                                        preprocessed_name=preprocessed_name)
doublons_counter = 0 
doublons_replacements = {}
for key in doublons:
    doublon_list = doublons[key]
    if len(doublon_list) >= 2:
        doublons_counter += 1
        print("\""+ key + "\"")
        replacement = doublon_list[0] # choosing 1st item of list because all the choices are equivalent
        print("  ", doublon_list[0].replace("http://schema.org/", " "))
        for i in range(1,len(doublon_list)):
            doublons_replacements[doublon_list[i]] = replacement
            print("  ", doublon_list[i].replace("http://schema.org/", " "))
print("doublons", doublons_counter)
len(publishers_bnf_constellations_dict)

"ankama"
    Ankama
    Ankamadition
"bamboo"
    Bamboo
    Bamboodition
"cadex"
    Cadex
    Cadexditions
"delcourt"
    Delcourt
    GroupeDelcourt
"denoel"
    Denoël
    Denöel
"ekare"
    Ekaré
    Ékaré
"fleurus"
    Fleurus
    FleurusEditions
"grund"
    Gründ
    Grund
    LibrairieGrnd
"hugo"
    Hugo
    HugoetCie
"lagapante"
    LAgapante
    lAgapanteCie
"larche"
    LArche
    LArcheEditeur
"lemeac"
    Leméac
    Lmac
"memo"
    MeMo
    MéMo
"palette"
    Palette
    palette
"pika"
    Pika
    Pikadition
"xyz"
    XYZ_Éditeur
    XYZ
"zebulo"
    Zébulo
    Zebuloditions
"etre"
    Être_Éditions
    tre
    Etre
"lansman"
    Lansman
    Lansmanditeur
"samir"
    Samir
    Samr
"flammarion"
    Flammarion
    Flammarionditeur
"duchene"
    Éditions_du_Chêne
    duChne
"123soleil"
    Éditions_123_Soleil
    1_2_3_Soleil
"lelievredemars"
    Le_Lièvre_de_Mars
    Le_lièvre_de_Mars
"petitsgenies"
    Petits_Génies
    Petits_génies
"dupepin"
    Éditions_du_Pépin
    d

2239

In [5]:
doublons_replacements

{rdflib.term.URIRef('http://schema.org/Ankamadition'): rdflib.term.URIRef('http://schema.org/Ankama'),
 rdflib.term.URIRef('http://schema.org/Bamboodition'): rdflib.term.URIRef('http://schema.org/Bamboo'),
 rdflib.term.URIRef('http://schema.org/Cadexditions'): rdflib.term.URIRef('http://schema.org/Cadex'),
 rdflib.term.URIRef('http://schema.org/GroupeDelcourt'): rdflib.term.URIRef('http://schema.org/Delcourt'),
 rdflib.term.URIRef('http://schema.org/Denöel'): rdflib.term.URIRef('http://schema.org/Denoël'),
 rdflib.term.URIRef('http://schema.org/Ékaré'): rdflib.term.URIRef('http://schema.org/Ekaré'),
 rdflib.term.URIRef('http://schema.org/FleurusEditions'): rdflib.term.URIRef('http://schema.org/Fleurus'),
 rdflib.term.URIRef('http://schema.org/Grund'): rdflib.term.URIRef('http://schema.org/Gründ'),
 rdflib.term.URIRef('http://schema.org/LibrairieGrnd'): rdflib.term.URIRef('http://schema.org/Gründ'),
 rdflib.term.URIRef('http://schema.org/HugoetCie'): rdflib.term.URIRef('http://schema.or

In [ ]:
graph_bnf = Graph()
graph_bnf.parse("final_datasets/bnf_auteursURI.ttl", format="turtle")


In [6]:

graph_constellations = Graph()
graph_constellations.parse("final_datasets/constellations_auteursURI.ttl", format="turtle")

<Graph identifier=N8e1686f1c0b04f90bbe7bc58e021c55d (<class 'rdflib.graph.Graph'>)>

In [ ]:
graph_bnf.subjects(RDF.type, utils.schema.Book)[0]

In [17]:
for publisher_doublon in doublons_replacements:

    for publisher in publishers_bnf_constellations_graph.subjects(RDF.type, SCHEMA.Publisher):
        if publisher == publisher_doublon:
            publishers_bnf_constellations_graph.remove((publisher, SCHEMA.name, None)) # remove attributes before reference so reference doesn't become orphan in graph
            publishers_bnf_constellations_graph.remove((publisher, RDF.type, None))
            print(publisher)
    for bnf_book in graph_bnf.subjects(RDF.type, utils.schema.Book):
        book_data = utils.extract_data_bnf(graph_bnf, bnf_book)
        book_publisher = book_data.publisher
        
        if book_publisher == str(publisher_doublon):
            graph_bnf.remove((bnf_book, SCHEMA.publisher, None))
            graph_bnf.add((bnf_book, SCHEMA.publisher, doublons_replacements[publisher_doublon]))

    for constellations_book in graph_constellations.subjects(RDF.type, utils.schema.Book):
        book_data = utils.extract_data_bnf(graph_constellations, constellations_book)
        book_publisher = book_data.publisher
        
        if book_publisher == str(publisher_doublon):
            print(book_publisher)
            graph_constellations.remove((constellations_book, SCHEMA.publisher, None))
            graph_constellations.add((constellations_book, SCHEMA.publisher, doublons_replacements[publisher_doublon]))

http://schema.org/Ankamadition
http://schema.org/Bamboodition
http://schema.org/Cadexditions
http://schema.org/GroupeDelcourt
http://schema.org/Denöel
http://schema.org/Ékaré
http://schema.org/FleurusEditions
http://schema.org/Grund
http://schema.org/LibrairieGrnd
http://schema.org/HugoetCie
http://schema.org/lAgapanteCie
http://schema.org/LArcheEditeur
http://schema.org/Lmac
http://schema.org/MéMo
http://schema.org/palette
http://schema.org/Pikadition
http://schema.org/XYZ
http://schema.org/Zebuloditions
http://schema.org/tre
http://schema.org/Etre
http://schema.org/Lansmanditeur
http://schema.org/Samr
http://schema.org/Flammarionditeur
http://schema.org/duChne
http://schema.org/1_2_3_Soleil
http://schema.org/Le_lièvre_de_Mars
http://schema.org/Petits_génies
http://schema.org/duPpin
http://schema.org/Golette
http://schema.org/Delachaux_et_Niestle
http://schema.org/Motus
http://schema.org/Éditions_Quatre_Fleuves
http://schema.org/La_poule_qui_pond
http://schema.org/DisneyHyperion_Books

In [23]:
fichier = open("../final_datasets/bnf_NoDuplicatePublisher.ttl","wb")
graph_bnf.serialize(fichier) 
fichier.close()

In [26]:
fichier = open("../final_datasets/constellations_NoDuplicatePublisher.ttl","wb")
graph_constellations.serialize(fichier) 
fichier.close()

In [18]:
fichier = open("../final_datasets/publisher_BNF_Constellation_NoDuplicate.ttl","wb")
publishers_bnf_constellations_graph.serialize(fichier) 
fichier.close()

In [ ]:
len(doublons)

58

In [ ]:
# assert(doublons == ['memo', 'ekare', 'denoel', 'le lievre de mars', 'palette', 'disneyhyperion books', 'petits genies', 'delachaux et niestle', 'la poule qui pond', 'motus', 'quatre fleuves', 'format', 'rue de lechiquier', 'eyrolles', 'ceres', 'pika', 'etre', 'grenouille', 'du beryl', 'ankama', 'du chene', 'dar alsaqi', 'grund', 'bamboo', 'dar aladab', 'edicef', 'du pepin', 'pemf', 'zebulo', 'etre', '', 'fleurus', 'cadex', 'dar alhadaiq', 'black box', 'goelette', 'koine', 'bld', 'samir', 'tsipika', '', 'lemeac', 'ndze', 'dar alhadaiq', 'bartholdi', 'h2t', '1018'])

In [ ]:
import Levenshtein

def find_top_x_similar_strings_levenshtein(string_list, target_string):
    distances = []
    for string in string_list:
        distance = Levenshtein.distance(string, target_string)
        if distance <= 8:
            distances.append((string, distance))
    distances.sort(key=lambda x: x[1])
    if distances:
         return distances[:5]
    else: 
        return None

In [ ]:
len(publishers_btlf_graph)

2044

In [ ]:
direct_match_counter = 0
levenstein_match_counter = 0
btlf_publishers_dict = {}
matches_dict = {}
non_matches_dict = {}
doublons_btlf = []
for publisher in publishers_btlf_graph.subjects(RDF.type, SCHEMA.Publisher):
    print(publisher)
    raw_name = str(publishers_btlf_graph.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)

    if preprocessed_name in btlf_publishers_dict.keys():
        doublons_btlf.append(preprocessed_name)
        
    btlf_publishers_dict[preprocessed_name] = utils.Publisher(source="btlf", 
                                                uri=publisher, 
                                                raw_name=raw_name, 
                                                preprocessed_name=preprocessed_name)

    if preprocessed_name in publishers_bnf_constellations_dict.keys():
        direct_match_counter +=1
        # print(preprocessed_name)
        matches_dict[preprocessed_name] = [btlf_publishers_dict[preprocessed_name], publishers_bnf_constellations_dict[preprocessed_name]]
    else:
        matched_key = find_top_x_similar_strings_levenshtein(publishers_bnf_constellations_dict.keys(), preprocessed_name)
        if matched_key:
            levenstein_match_counter += 1
            print("\""+ raw_name+ "\"")
            for key in matched_key:
                print("  ", publishers_bnf_constellations_dict[key[0]].uri.replace("http://schema.org/", " "))
        else:
            non_matches_dict[preprocessed_name] = publisher
print()
print("direct", direct_match_counter, "/", len(btlf_publishers_dict.keys()))
print("levenstein", levenstein_match_counter, "/", len(btlf_publishers_dict.keys()))
print("sum = ", direct_match_counter + levenstein_match_counter)

http://schema.org/0
http://schema.org/1
http://schema.org/10
http://schema.org/100
http://schema.org/1000
http://schema.org/1001
http://schema.org/1002
"Alter Comics"
    AH_Comics
    Blisscomics
    DC_Comics
    Alternatives
    404comics
http://schema.org/1003
"Didier-Erudition"
    Didier_Jeunesse
    AMichelducation
http://schema.org/1004
"Éditions Chouette"
    Clochette
    Caurette
    Cheyne
    Palette
    Hachette
http://schema.org/1005
"Les éditions d'Avallon"
    Pavilion
    Dalloz
    Amalion
    Hamilton
    Magellan___Cie
http://schema.org/1006
"Éditions Logiques"
    Tropiques
    LOlivier
    Utopique
    afriqueeditions
    BJacques
http://schema.org/1007
http://schema.org/1008
"Éditions du Vermillon"
    Vermillon
    Tourbillon
    duCercil
    Du_Centurion
    Hamilton
http://schema.org/1009
http://schema.org/101
http://schema.org/1010
http://schema.org/1011
"Age d'homme"
    delHomme
    lagomme
    Petit_Homme
    ADEME
    zedcom
http://schema.org/1012
"Boome

In [ ]:
len(non_matches_dict)

32

In [ ]:
non_matches_dict

{'canopecrdpdelacademiedeparis': rdflib.term.URIRef('http://schema.org/128'),
 'bretagneenvironnement': rdflib.term.URIRef('http://schema.org/150'),
 'maitrisedeshautsdeseine': rdflib.term.URIRef('http://schema.org/232'),
 'sncfdirectiondelarecherchedepartementmarketingetdeveloppement': rdflib.term.URIRef('http://schema.org/287'),
 'radiofranceinternationale': rdflib.term.URIRef('http://schema.org/298'),
 'canopecrdpdelacademiedaixmarseille': rdflib.term.URIRef('http://schema.org/301'),
 'associationculturellefranceamerique': rdflib.term.URIRef('http://schema.org/314'),
 'galeriebarbier&mathon': rdflib.term.URIRef('http://schema.org/316'),
 'museeduluxembourgdeparis': rdflib.term.URIRef('http://schema.org/334'),
 'lememorialdecaen': rdflib.term.URIRef('http://schema.org/363'),
 'chateaudeversailles': rdflib.term.URIRef('http://schema.org/395'),
 'mondeglobalnouvelles': rdflib.term.URIRef('http://schema.org/400'),
 'bibliothequelecturedeveloppement': rdflib.term.URIRef('http://schema.or